In [ ]:
import nltk
nltk.download('punkt')       
nltk.download('stopwords') 
nltk.download('punkt_tab')  
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import numpy as np
import pandas as pd
from helper_functions import load_clean_csv
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Load in Recall data

# Paths to all three files
recall_files = [
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_ArtsandCrafts.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Riding_Toys.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Toys.csv"
]

recall_dfs = [load_clean_csv(path) for path in recall_files]
recalls_df = pd.concat(recall_dfs, ignore_index=True)



In [10]:
# Load in reviews data
reviews_df = pd.read_pickle('reviews_raw.pkl')
reviews_df['asin'].nunique()

624792

In [15]:
# initialize model to create embeddings on incident description text
model = SentenceTransformer('all-MiniLM-L6-v2')

In [17]:
# from the recalls data, embed the incident description
incident_desc_embedding = model.encode(recalls_df['Incident Description'].to_list())


In [ ]:
# drop na in reviewtext, asin and summary column
# TODO: should we be dropping duplicates?
reviews_model_df = reviews_df[['asin', 'reviewText', 'summary' ,'overall','vote']].copy()
reviews_model_df = reviews_model_df.dropna(subset=['asin','reviewText', 'summary'])

In [22]:
# strip possible leading or trailing white space
reviews_model_df = reviews_model_df[reviews_model_df['reviewText'].str.strip() != '']
reviews_model_df = reviews_model_df[reviews_model_df['summary'].str.strip() != '']

In [ ]:
# embed the reviewsText and summary columns line by line
reviews_model_df['reviews_embedding'] = reviews_model_df['reviewText'].apply(lambda x: model.encode(x, show_progress_bar=False))
reviews_model_df['summary_embedding'] = reviews_model_df['summary'].apply(lambda x: model.encode(x, show_progress_bar=False))

In [ ]:
# Aggregate embeddings per ASIN (mean)
final_model_df['embedding'] = final_model_df['embedding'].apply(lambda x: np.array(x))
asin_embeddings = final_model_df.groupby('asin')['embedding'].apply(lambda x: np.mean(np.stack(x), axis=0)).reset_index()


In [ ]:
# other features to consider
# avg_review_length, avg_sentence_count, punctuation_density
# capture sentiment like tone or emotion not fully captured in embedding: avg_sentiment_score

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Apply VADER to each review
def get_sentiment_scores(text):
    return analyzer.polarity_scores(text)

sentiment_df = final_model_df['reviewText'].apply(get_sentiment_scores).apply(pd.Series)

# Add to main dataframe
final_model_df = pd.concat([final_model_df, sentiment_df], axis=1)

# Group by ASIN and aggregate
asin_sentiment = final_model_df.groupby('asin')[['compound', 'pos', 'neu', 'neg']].mean().reset_index()

# (Optional) Add std deviation if you want:
sentiment_std = final_model_df.groupby('asin')['compound'].std().reset_index().rename(columns={'compound': 'compound_std'})
asin_sentiment = asin_sentiment.merge(sentiment_std, on='asin', how='left')



In [ ]:
asin_sentiment

In [ ]:
asin_embeddings

In [ ]:
design_matrix_test = pd.merge(asin_embeddings, asin_sentiment, on='asin')

In [ ]:
design_matrix_test

In [ ]:
reviews_df['asin']

4694586    B00WJ1OQC6
7550598    B00S0WXWI2
4640828    B00VJMDZHS
6539636    B005ZI6ZH4
4420882    B00SJP73H6
Name: asin, dtype: object

In [104]:
sample_asins = ['B00WJ1OQC6','B00S0WXWI2', 'B00VJMDZHS','B005ZI6ZH4', 'B00SJP73H6'  ]

In [106]:
reviews_df.loc[reviews_df['asin'].isin(sample_asins)].to_clipboard()